In [1]:

NUM_EPOCHS = 40
NUM_TRIALS = 30

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import ast
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (recall_score, precision_score, f1_score,
                             confusion_matrix, classification_report,
                             roc_auc_score)
from sklearn.utils.class_weight import compute_class_weight
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import WeightedRandomSampler
from torch.cuda.amp import GradScaler, autocast
import optuna
import json
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

SAVE_FOLDER = "./"
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)
    print(f"Created folder: {SAVE_FOLDER}")

TRAIN_CSV = "./iemocap_train.csv"
TEST_CSV = "./iemocap_test.csv"
CHECKPOINT_PATH = os.path.join(SAVE_FOLDER, "lstm_best_model.pth")
HYPERPARAMS_PATH = os.path.join(SAVE_FOLDER, "best_hyperparameters.json")

for csv_file in [TRAIN_CSV, TEST_CSV]:
    if not os.path.exists(csv_file):
        raise FileNotFoundError(f"File '{csv_file}' not found in '{SAVE_FOLDER}'. Please upload it.")
    print(f"File '{csv_file}' found. Size: {os.path.getsize(csv_file)} bytes")

SEED = 4321
INPUT_SIZE = 29
NUM_WORKERS = 14

torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
else:
    print("Single GPU or CPU detected")

# -----------------------------
# Data Loading and Preprocessing
# -----------------------------
def load_csv_robust(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
        print(f"Successfully loaded '{file_path}' with {len(df)} rows.")
        return df
    except pd.errors.ParserError as e:
        print(f"ParserError in '{file_path}': {e}")
        df = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='warn')
        print(f"Loaded '{file_path}' with skipped lines: {len(df)} rows.")
        return df

df_train = load_csv_robust(TRAIN_CSV)
df_test = load_csv_robust(TEST_CSV)

def extract_features(row):
    try:
        features_list = ast.literal_eval(row["audio_features"])
        return np.array(features_list, dtype=np.float32)
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing 'audio_features' in row: {row}. Error: {e}")
        return np.zeros((1, INPUT_SIZE), dtype=np.float32)

df_train["Features"] = df_train.apply(extract_features, axis=1)
df_train["lengths"] = df_train["Features"].apply(lambda x: x.shape[0])
df_test["Features"] = df_test.apply(extract_features, axis=1)
df_test["lengths"] = df_test["Features"].apply(lambda x: x.shape[0])

le = LabelEncoder()
df_train["Label"] = le.fit_transform(df_train["emotion"])
df_test["Label"] = le.transform(df_test["emotion"])
num_classes = len(le.classes_)

class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(df_train["Label"]), y=df_train["Label"])
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# -----------------------------
# Dataset and DataLoader
# -----------------------------
class IEMOCAPDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        features = torch.tensor(row["Features"], dtype=torch.float32)
        label = torch.tensor(row["Label"], dtype=torch.long)
        length = row["lengths"]
        return features, label, length

def collate_fn(batch):
    features, labels, lengths = zip(*batch)
    features = [f if isinstance(f, torch.Tensor) else torch.tensor(f, dtype=torch.float32) for f in features]
    lengths = torch.tensor(lengths, dtype=torch.long)
    features_padded = pad_sequence(features, batch_first=True, padding_value=0.0)
    labels = torch.tensor(labels, dtype=torch.long)
    return features_padded, labels, lengths

class_counts = df_train["Label"].value_counts().sort_index()
sample_weights = 1.0 / class_counts[df_train["Label"]].values
sample_weights = torch.tensor(sample_weights, dtype=torch.double)
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

train_dataset = IEMOCAPDataset(df_train)
test_dataset = IEMOCAPDataset(df_test)

# -----------------------------
# Attention Mechanism
# -----------------------------
class Attention(nn.Module):
    def __init__(self, hidden_size, temperature=1.0):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.score = nn.Linear(hidden_size, 1, bias=False)
        self.temperature = temperature

    def forward(self, lstm_output, lengths):
        attn_weights = torch.tanh(self.attn(lstm_output))
        attn_scores = self.score(attn_weights).squeeze(-1)
        mask = torch.arange(lstm_output.size(1), device=lstm_output.device).unsqueeze(0) >= lengths.to(lstm_output.device).unsqueeze(1)
        attn_scores.masked_fill_(mask, float('-inf'))
        attn_weights = torch.softmax(attn_scores / self.temperature, dim=1)
        context = (attn_weights.unsqueeze(-1) * lstm_output).sum(dim=1)
        return context, attn_weights

# -----------------------------
# Model Definition
# -----------------------------
class LSTMAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout, temperature=1.0):
        super(LSTMAttention, self).__init__()
        self.lstm_emotion = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=dropout)
        self.attention = Attention(hidden_size, temperature=temperature)
        self.fc_emotion = nn.Linear(hidden_size * 2, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, lengths):
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _ = self.lstm_emotion(packed)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)
        context, attn_weights = self.attention(out, lengths)
        context = self.dropout(context)
        emotion_out = self.fc_emotion(context)
        return emotion_out, attn_weights

# -----------------------------
# Focal Loss
# -----------------------------
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha if alpha is not None else 1.0
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = (self.alpha[targets] * (1 - pt) ** self.gamma * ce_loss)
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# -----------------------------
# Evaluation Metrics Function
# -----------------------------
def compute_metrics(labels, preds, outputs, num_classes):
    metrics = {}
    metrics['war'] = recall_score(labels, preds, average='micro')
    metrics['uar'] = recall_score(labels, preds, average='macro')
    metrics['precision_macro'] = precision_score(labels, preds, average='macro')
    metrics['f1_macro'] = f1_score(labels, preds, average='macro')
    metrics['precision_weighted'] = precision_score(labels, preds, average='weighted')
    metrics['f1_weighted'] = f1_score(labels, preds, average='weighted')
    metrics['conf_matrix'] = confusion_matrix(labels, preds)
    if num_classes > 2:
        try:
            metrics['roc_auc_ovr'] = roc_auc_score(labels, outputs, multi_class='ovr')
        except:
            metrics['roc_auc_ovr'] = None
    else:
        metrics['roc_auc'] = roc_auc_score(labels, outputs[:, 1])
    return metrics

# -----------------------------
# Objective Function for Optuna
# -----------------------------
def objective(trial):
    print(f"\nStarting Optuna Trial {trial.number} at {pd.Timestamp.now()}")

    hidden_size = trial.suggest_categorical('hidden_size', [32, 64, 128, 256])
    num_layers = trial.suggest_int('num_layers', 1, 4)
    dropout = trial.suggest_float('dropout', 0.2, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-3, log=True)
    focal_gamma = trial.suggest_float('focal_gamma', 2.0, 4.0)
    temperature = trial.suggest_float('temperature', 1.0, 5.0)
    weight_decay = trial.suggest_float('weight_decay', 1e-3, 1e-1, log=True)
    max_norm = trial.suggest_float('max_norm', 0.1, 1.0)
    scheduler_factor = trial.suggest_float('scheduler_factor', 0.1, 0.5)
    scheduler_patience = trial.suggest_int('scheduler_patience', 3, 5)

    params = {
        'hidden_size': hidden_size,
        'num_layers': num_layers,
        'dropout': dropout,
        'batch_size': batch_size,
        'learning_rate': learning_rate,
        'focal_gamma': focal_gamma,
        'temperature': temperature,
        'weight_decay': weight_decay,
        'max_norm': max_norm,
        'scheduler_factor': scheduler_factor,
        'scheduler_patience': scheduler_patience
    }
    print("Hyperparameters:")
    for param, value in params.items():
        print(f"  {param}: {value}")

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, sampler=sampler, num_workers=NUM_WORKERS,
        collate_fn=collate_fn, pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS,
        collate_fn=collate_fn, pin_memory=True
    )


    model = LSTMAttention(INPUT_SIZE, hidden_size, num_layers, num_classes, dropout, temperature)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model = model.to(device)
    model = torch.compile(model)

    emotion_criterion = FocalLoss(alpha=class_weights.to(device), gamma=focal_gamma)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=scheduler_factor, patience=scheduler_patience)
    scaler = GradScaler()

    best_uar = 0.0
    for epoch in range(NUM_EPOCHS):
        model.train()
        epoch_loss = 0.0
        train_correct = 0
        train_total = 0

        for features, label, lengths in train_loader:
            features = features.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            with autocast():
                emotion_out, _ = model(features, lengths)
                loss = emotion_criterion(emotion_out, label)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()
            _, preds = torch.max(emotion_out, 1)
            train_correct += (preds == label).sum().item()
            train_total += label.size(0)


        model.eval()
        all_labels, all_preds, all_outputs = [], [], []
        val_loss = 0.0
        with torch.no_grad():
            for features, label, lengths in test_loader:
                features = features.to(device)
                label = label.to(device)
                with autocast():
                    emotion_out, _ = model(features, lengths)
                    loss = emotion_criterion(emotion_out, label)
                val_loss += loss.item()
                _, preds = torch.max(emotion_out, 1)
                all_labels.extend(label.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())
                all_outputs.extend(torch.softmax(emotion_out, dim=1).cpu().numpy())

        all_labels = np.array(all_labels)
        all_preds = np.array(all_preds)
        all_outputs = np.array(all_outputs)
        metrics = compute_metrics(all_labels, all_preds, all_outputs, num_classes)
        uar = metrics['uar']
        scheduler.step(uar)

        if uar > best_uar:
            best_uar = uar

    print(f"\nFinished Optuna Trial {trial.number} at {pd.Timestamp.now()}")
    print(f"Best UAR: {best_uar:.4f}")
    print("-" * 50)
    return best_uar

# -----------------------------
# Run Optuna Optimization
# -----------------------------
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=NUM_TRIALS)

print("\nBest Hyperparameters:")
print(study.best_params)
print(f"Best UAR: {study.best_value:.4f}")

with open(HYPERPARAMS_PATH, 'w') as f:
    json.dump({
        'best_params': study.best_params,
        'best_uar': study.best_value
    }, f, indent=4)
print(f"Best hyperparameters saved to: {HYPERPARAMS_PATH}")

File './iemocap_train.csv' found. Size: 441434383 bytes
File './iemocap_test.csv' found. Size: 114816810 bytes
Using device: cuda
Single GPU or CPU detected
Successfully loaded './iemocap_train.csv' with 5904 rows.
Successfully loaded './iemocap_test.csv' with 1476 rows.


[I 2025-03-21 20:03:38,075] A new study created in memory with name: no-name-a36c9def-5c2b-4044-b564-f57c6098769b



Starting Optuna Trial 0 at 2025-03-21 20:03:38.075815
Hyperparameters:
  hidden_size: 256
  num_layers: 4
  dropout: 0.24547619140839227
  batch_size: 32
  learning_rate: 0.0003345522177503116
  focal_gamma: 3.3709896152295458
  temperature: 2.3774866674463504
  weight_decay: 0.048351324370537664
  max_norm: 0.9970966580791061
  scheduler_factor: 0.44295720436939146
  scheduler_patience: 5


[I 2025-03-21 20:17:36,094] Trial 0 finished with value: 0.49138750799092357 and parameters: {'hidden_size': 256, 'num_layers': 4, 'dropout': 0.24547619140839227, 'batch_size': 32, 'learning_rate': 0.0003345522177503116, 'focal_gamma': 3.3709896152295458, 'temperature': 2.3774866674463504, 'weight_decay': 0.048351324370537664, 'max_norm': 0.9970966580791061, 'scheduler_factor': 0.44295720436939146, 'scheduler_patience': 5}. Best is trial 0 with value: 0.49138750799092357.



Finished Optuna Trial 0 at 2025-03-21 20:17:36.091219
Best UAR: 0.4914
--------------------------------------------------

Starting Optuna Trial 1 at 2025-03-21 20:17:36.095991
Hyperparameters:
  hidden_size: 128
  num_layers: 4
  dropout: 0.4787639973055768
  batch_size: 16
  learning_rate: 0.0009315376652218845
  focal_gamma: 3.4226422513903634
  temperature: 2.430664820085964
  weight_decay: 0.06948020347732659
  max_norm: 0.22426911461659618
  scheduler_factor: 0.26409583188907426
  scheduler_patience: 3


...


Finished Optuna Trial 27 at 2025-03-22 05:09:16.402899
Best UAR: 0.4613
--------------------------------------------------

Starting Optuna Trial 28 at 2025-03-22 05:09:16.408641
Hyperparameters:
  hidden_size: 256
  num_layers: 2
  dropout: 0.22902650040397707
  batch_size: 8
  learning_rate: 0.0006319257640538455
  focal_gamma: 2.1771255166892662
  temperature: 1.0234125630247206
  weight_decay: 0.03018703975183937
  max_norm: 0.9217844350515917
  scheduler_factor: 0.1979982256696693
  scheduler_patience: 5


[I 2025-03-22 05:38:26,095] Trial 28 finished with value: 0.4837262981475506 and parameters: {'hidden_size': 256, 'num_layers': 2, 'dropout': 0.22902650040397707, 'batch_size': 8, 'learning_rate': 0.0006319257640538455, 'focal_gamma': 2.1771255166892662, 'temperature': 1.0234125630247206, 'weight_decay': 0.03018703975183937, 'max_norm': 0.9217844350515917, 'scheduler_factor': 0.1979982256696693, 'scheduler_patience': 5}. Best is trial 0 with value: 0.49138750799092357.



Finished Optuna Trial 28 at 2025-03-22 05:38:26.091657
Best UAR: 0.4837
--------------------------------------------------

Starting Optuna Trial 29 at 2025-03-22 05:38:26.096447
Hyperparameters:
  hidden_size: 128
  num_layers: 4
  dropout: 0.29190236852836393
  batch_size: 32
  learning_rate: 0.0001810129708132406
  focal_gamma: 3.391996625787396
  temperature: 2.31183099941698
  weight_decay: 0.013648061643674195
  max_norm: 0.24619847089196523
  scheduler_factor: 0.2942518307224353
  scheduler_patience: 3


[I 2025-03-22 05:51:31,010] Trial 29 finished with value: 0.4619558801840586 and parameters: {'hidden_size': 128, 'num_layers': 4, 'dropout': 0.29190236852836393, 'batch_size': 32, 'learning_rate': 0.0001810129708132406, 'focal_gamma': 3.391996625787396, 'temperature': 2.31183099941698, 'weight_decay': 0.013648061643674195, 'max_norm': 0.24619847089196523, 'scheduler_factor': 0.2942518307224353, 'scheduler_patience': 3}. Best is trial 0 with value: 0.49138750799092357.



Finished Optuna Trial 29 at 2025-03-22 05:51:31.004866
Best UAR: 0.4620
--------------------------------------------------

Best Hyperparameters:
{'hidden_size': 256, 'num_layers': 4, 'dropout': 0.24547619140839227, 'batch_size': 32, 'learning_rate': 0.0003345522177503116, 'focal_gamma': 3.3709896152295458, 'temperature': 2.3774866674463504, 'weight_decay': 0.048351324370537664, 'max_norm': 0.9970966580791061, 'scheduler_factor': 0.44295720436939146, 'scheduler_patience': 5}
Best UAR: 0.4914
Best hyperparameters saved to: ./best_hyperparameters.json
